In [4]:
from LAC import LAC

C:\Anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable


In [5]:
lac = LAC(mode = 'rank')

In [5]:
text = u'西甲的第一名是谁' # Who is No. 1 in La Liga?
lac_result = lac.run(text)
lac_result

[['西甲', '的', '第一名', '是', '谁'], ['nz', 'u', 'm', 'v', 'r'], [3, 0, 2, 0, 1]]

In [6]:
texts = [u'马德里竞技俱乐部在西甲里面排名多少', '皇家贝蒂斯足球俱乐部的排名和皇家马德里俱乐部的排名那个比较好']
# 'How much does Atlético Madrid rank in La Liga', 'Which is better, the ranking of Real Betis Football Club or the ranking of Real Madrid Club'
lac_results = lac.run(texts)
lac_results

[[['马德里竞技俱乐部', '在', '西甲', '里面', '排名', '多少'],
  ['ORG', 'p', 'nz', 'f', 'v', 'r'],
  [3, 0, 3, 1, 2, 1]],
 [['皇家贝蒂斯足球俱乐部', '的', '排名', '和', '皇家马德里俱乐部', '的', '排名', '那个', '比较好'],
  ['ORG', 'u', 'vn', 'c', 'ORG', 'u', 'vn', 'r', 'a'],
  [2, 0, 2, 0, 3, 0, 2, 1, 2]]]

In [7]:
text = '塞维利亚俱乐部的出生年月是什么时候？' # When was Sevilla FC born?
lac_result = lac.run(text)
lac_result

[['塞维利亚俱乐部', '的', '出生', '年月', '是', '什么时候', '？'],
 ['ORG', 'u', 'vn', 'n', 'v', 'n', 'w'],
 [3, 0, 2, 2, 0, 2, 0]]

In [6]:
import synonyms
# print('国籍: ', synonyms.nearby("国籍")) # nationality
print(synonyms.compare('简称','绰号')) # 'abbreviation', 'nickname'

[jieba] default dict file path ..\data\vocab.txt
[jieba] default dict file path ..\data\vocab.txt
[jieba] load default dict ..\data\vocab.txt ...
[jieba] load default dict ..\data\vocab.txt ...
>> Synonyms load wordseg dict [C:\Anaconda3\lib\site-packages\synonyms\data\vocab.txt] ... 
>> Synonyms on loading stopwords [C:\Anaconda3\lib\site-packages\synonyms\data\stopwords.txt] ...
[Synonyms] on loading vectors [C:\Anaconda3\lib\site-packages\synonyms\data\words.vector.gz] ...


C:\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
C:\Anaconda3\lib\site-packages\synonyms\word2vec.py:175: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  weights = fromstring(fin.read(binary_len), dtype=REAL)


0.197


In [7]:
def get_closest_prop(n):
    n_list = [
        '主要荣誉',
        '位于',
        '国籍',
        '容纳',
        '成立时间',
        '球衣号码',
        '球迷昵称',
        '生日',
        '简称',
        '绰号',
        '职位'
    ]
    n_sim = []
    for i in n_list:
        n_sim.append(synonyms.compare(n, i))
    # print(n_sim)
    max_value = max(n_sim)
    idx = n_sim.index(max_value)
    return n_list[idx], max_value

In [8]:
get_closest_prop('名字')

('绰号', 0.893)

In [45]:
def get_entity_info(sentence):
    lac_res = lac.run(sentence)
    # print(lac_res)
    max_sim = -1
    max_n = ''
    # print(list(enumerate(lac_res[2])))
    for index, i in enumerate(lac_res[2]):
        if i > 0:
            # print(index)
            tmp_n, tmp_sim = get_closest_prop(lac_res[0][index])
            if tmp_sim > max_sim:
                max_sim = tmp_sim
                max_n = tmp_n
    entity = ''
    for index, i in enumerate(lac_res[1]):
        if i == 'PER' or i == 'LOC' or i == 'ORG':
            entity = lac_res[0][index]
    # print(entity, max_n)
    SPARQL_SEN = 'MATCH (a:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#' + entity + '"})\nRETURN a.ns0__' + max_n
    print(SPARQL_SEN)
    return SPARQL_SEN

In [46]:

q1=get_entity_info('皇家马略卡足球俱乐部的昵称是什么')
print(q1)

MATCH (a:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#皇家马略卡足球俱乐部"})
RETURN a.ns0__绰号
MATCH (a:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#皇家马略卡足球俱乐部"})
RETURN a.ns0__绰号


In [12]:
def get_closest_relation(r):
    r_list = ["主要竞争对手", "就任主席", "所属球队", "执教", "球队主场", "赞助商", "队长"]
    r_sim = []
    for i in r_list:
        r_sim.append(synonyms.compare(r, i))
    # print(r_sim)
    max_value = max(r_sim)
    idx = r_sim.index(max_value)
    return r_list[idx], max_value

In [36]:
def get_entity_relation(sentence):
    lac_res = lac.run(sentence)
    # print(lac_res)
    max_sim = -1
    max_r = ''
    # print(list(enumerate(lac_res[2])))
    for index, i in enumerate(lac_res[2]):
        if i > 0:
            # print(index)
            tmp_n, tmp_sim = get_closest_relation(lac_res[0][index])
            if tmp_sim > max_sim:
                max_sim = tmp_sim
                max_r = tmp_n
    entity = ''
    for index, i in enumerate(lac_res[1]):
        if i == 'PER' or i == 'LOC' or i == 'ORG':
            entity = lac_res[0][index]
    # print(entity, max_r)
    SPARQL_SEN ='MATCH (a:`ns0__主教练`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#' + entity + '"}) - [:`ns0__'+ max_r +'`]->(b) \nRETURN b'
    print(SPARQL_SEN)
    return SPARQL_SEN

In [37]:
q2=get_entity_relation('卡尔洛·安切洛蒂在哪个球队执教')
print(q2)

MATCH (a:`ns0__主教练`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#卡尔洛·安切洛蒂"}) - [:`ns0__执教`]->(b) 
RETURN b
MATCH (a:`ns0__主教练`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#卡尔洛·安切洛蒂"}) - [:`ns0__执教`]->(b) 
RETURN b


In [38]:
def get_entity_relation_value(sentence):
    lac_res = lac.run(sentence)
    # print(lac_res)
    max_sim = -1
    max_r = ''
    # print(list(enumerate(lac_res[2])))
    for index, i in enumerate(lac_res[2]):
        if i > 0:
            # print(index)
            tmp_n, tmp_sim = get_closest_relation(lac_res[0][index])
            if tmp_sim > max_sim:
                max_sim = tmp_sim
                max_r = tmp_n
    entity = ''
    for index, i in enumerate(lac_res[1]):
        if i == 'PER' or i == 'LOC' or i == 'ORG':
            entity = lac_res[0][index]
    # print(entity, max_r)
    SPARQL_SEN = 'MATCH (a:entity{name:"' + entity + '"}) - [r:'+ max_r +']->(b) \nRETURN r'
    print(SPARQL_SEN)
    return SPARQL_SEN

In [39]:
get_entity_relation_value('皇家马德里俱乐部的所有球员有哪些')


MATCH (a:entity{name:"皇家马德里俱乐部"}) - [r:主要竞争对手]->(b) 
RETURN r


'MATCH (a:entity{name:"皇家马德里俱乐部"}) - [r:主要竞争对手]->(b) \nRETURN r'

In [26]:
def get_entitys_better(sentence):
    lac_res = lac.run(sentence)
    # print(lac_res)
    max_sim = -1
    max_r = ''
    # print(list(enumerate(lac_res[2])))
    for index, i in enumerate(lac_res[2]):
        if i > 0:
            # print(index)
            tmp_n, tmp_sim = get_closest_relation(lac_res[0][index])
            if tmp_sim > max_sim:
                max_sim = tmp_sim
                max_r = tmp_n
    entity_list = []
    for index, i in enumerate(lac_res[1]):
        if i == 'PER' or i == 'LOC' or i == 'ORG':
            entity_list.append(lac_res[0][index])
    # print(entity_list, max_r)
    SPARQL_SEN = 'MATCH (a:entity1{name:"' + entity_list[0] + '"}),(b:entity2{name:"' + entity_list[1] + '"}) \nWHERE a.property>b.property\nRETURN a,b'
    # print(SPARQL_SEN)
    return SPARQL_SEN

In [27]:
get_entitys_better('皇家马德里俱乐部赛季排名是否比塞尔维亚俱乐部好？')

'MATCH (a:entity1{name:"皇家马德里俱乐部"}),(b:entity2{name:"塞尔维亚俱乐部"}) \nWHERE a.property>b.property\nRETURN a,b'

In [40]:
def get_relation_between(sentence):
    lac_res = lac.run(sentence)
    # print(lac_res)
    max_sim = -1
    max_r = ''
    # print(list(enumerate(lac_res[2])))
    for index, i in enumerate(lac_res[2]):
        if i > 0:
            # print(index)
            tmp_n, tmp_sim = get_closest_relation(lac_res[0][index])
            if tmp_sim > max_sim:
                max_sim = tmp_sim
                max_r = tmp_n
    entity_list = []
    for index, i in enumerate(lac_res[1]):
        if i == 'PER' or i == 'LOC' or i == 'ORG':
            entity_list.append(lac_res[0][index])
    # print(entity_list, max_r)
    SPARQL_SEN = 'MATCH (a:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#' + entity_list[0] + '"})-[r] -(b:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#' + entity_list[1] + '"})\nRETURN r'
    print(SPARQL_SEN)
    return SPARQL_SEN

In [41]:
q3=get_relation_between('皇家贝蒂斯足球俱乐部和塞维利亚足球俱乐部是什么关系？')
print(q3)

MATCH (a:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#皇家贝蒂斯足球俱乐部"})-[r] -(b:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#塞维利亚足球俱乐部"})
RETURN r
MATCH (a:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#皇家贝蒂斯足球俱乐部"})-[r] -(b:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#塞维利亚足球俱乐部"})
RETURN r


In [30]:
def get_better_entities(sentence):
    lac_res = lac.run(sentence)
    # print(lac_res)
    max_sim = -1
    max_r = ''
    # print(list(enumerate(lac_res[2])))
    for index, i in enumerate(lac_res[2]):
        if i > 0:
            # print(index)
            tmp_n, tmp_sim = get_closest_relation(lac_res[0][index])
            if tmp_sim > max_sim:
                max_sim = tmp_sim
                max_r = tmp_n
    entity_list = []
    for index, i in enumerate(lac_res[1]):
        if i == 'PER' or i == 'LOC' or i == 'ORG':
            entity_list.append(lac_res[0][index])
    # print(entity_list, max_r)
    SPARQL_SEN = 'MATCH (a:entity1{name:"' + entity_list[0] + '"}),(b:entity2})\nWHERE a.'+ max_r +' < b.'+ max_r +'\nRETURN b'
    print(SPARQL_SEN)
    return SPARQL_SEN

In [31]:
get_better_entities('有哪些比皇家马德里俱乐部赛季排名好的球队？')

MATCH (a:entity1{name:"皇家马德里俱乐部"}),(b:entity2})
WHERE a.执教 < b.执教
RETURN b


'MATCH (a:entity1{name:"皇家马德里俱乐部"}),(b:entity2})\nWHERE a.执教 < b.执教\nRETURN b'

In [42]:
# interface
lac = LAC(mode='rank')
aim_list = ['主要荣誉','位于','国籍','容纳','成立时间','球衣号码','球迷昵称','生日','简称','绰号','职位',"主要竞争对手", "就任主席", "所属球队", "执教", "球队主场", "赞助商", "队长"]
l_list = ['主要荣誉','位于','国籍','容纳','成立时间','球衣号码','球迷昵称','生日','简称','绰号','职位']
r_list = ["主要竞争对手", "就任主席", "所属球队", "执教", "球队主场", "赞助商", "队长"]
def question_part(s,aim_list):
    key_index = [] #重要词汇下标
    simi_list = [] #对比aim_list形似度表
    lac_result = lac.run(s)
    max_index = 0
    temp_list = []
    for i in lac_result[2]:
        if(i!=0):
            key_index.append(i)
    for i in range(len(aim_list)):
        for j in range(len(key_index)):
            temp_list.append(synonyms.compare(aim_list[i],lac_result[0][j]))
            simi_list.append(temp_list)
            temp_list = []
   #定位最相似下标
            temp = simi_list[0][0]
    for i in range(len(simi_list)):
        for j in range(len(simi_list[i])):
            if(simi_list[i][j]>temp):
                temp = simi_list[i][j]
                max_index = i
   # print(lac_result)
   # print(simi_list)
    return aim_list[max_index]

def which_template(q_part,sentence):
    temp = 0
    lac_res = lac.run(sentence)
    # print(lac_res)
    entity_list = []
    for index, i in enumerate(lac_res[1]):
        if i == 'PER' or i == 'LOC' or i == 'ORG':
            entity_list.append(lac_res[0][index])
    if(len(entity_list)>1):
        get_relation_between(sentence)#q3
    else:
        for i in range(len(l_list)):
            if(q_part == l_list[i]):
                get_entity_info(sentence)#q1
                temp = 1
        if(temp == 0):
            get_entity_relation(sentence)#q2

## test

In [47]:
#aim_list已固定，只需修改输入的sentence
sentence1 = '皇家马略卡足球俱乐部的昵称是什么'
sentence2 = '卡尔洛·安切洛蒂在哪个球队执教'
sentence3 = '皇家贝蒂斯足球俱乐部和塞维利亚足球俱乐部是什么关系？'
q_part = question_part(sentence1,aim_list)
which_template(q_part,sentence1)
q_part = question_part(sentence2,aim_list)
which_template(q_part,sentence2)
q_part = question_part(sentence3,aim_list)
which_template(q_part,sentence3)

MATCH (a:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#皇家马略卡足球俱乐部"})
RETURN a.ns0__绰号
MATCH (a:`ns0__主教练`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#卡尔洛·安切洛蒂"}) - [:`ns0__执教`]->(b) 
RETURN b
MATCH (a:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#皇家贝蒂斯足球俱乐部"})-[r] -(b:`ns0__足球俱乐部`{uri:"http://www.semanticweb.org/think/ontologies/Football.owl#塞维利亚足球俱乐部"})
RETURN r


In [48]:
import json
from py2neo import Graph,Node,Relationship,NodeMatcher,Subgraph

graph = Graph('http://localhost:7474', auth=("neo4j", "Lyc111lyc"))


In [ ]:
matcher = NodeMatcher(graph)

In [54]:
graph.run(q1)

"LosBermellones"


In [57]:
graph.run(q2)

{"identity": 1083,"labels": ["Resource","ns0__足球俱乐部", "owl__NamedIndividual"],"properties": {"ns0__简称": "皇马","ns0__主要荣誉": "西班牙足球甲级联赛冠军(34次)","ns0__位于": "西班牙首都马德里","ns0__成立时间": "1902年","ns0__球迷昵称": "美凌格","uri": "http://www.semanticweb.org/think/ontologies/Football.owl#皇家马德里足球俱乐部"}}


In [62]:
graph.run(q3)

{"identity": 1039,"start": 1278,"end": 1164, "type": "ns0__主要竞争对手","properties": {}}
{"identity": 912,"start": 1164,"end": 1278, "type": "ns0__主要竞争对手","properties": {}}


In [3]:

#matcher = NodeMatcher(graph)
#matcher.match().first()

In [2]:
#result=graph.run("MATCH (n) RETURN n").data()#读取neo4j数据库中的所有数据
#record=result[0] #读取表中的第一个数据
#record.get('n').get('uri') #假设数据库中有一个属性（或者说是列名字）是name


In [1]:
#graph.run(q1)#.data()